In [1]:
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

In [12]:
# Load the saved model
model = load_model('/content/ccmt5gb.h5')

In [13]:
# Load the CSV with the mapping of pests to solutions
csv_path = '/content/diseases_commercial_organic.csv'
solutions_df = pd.read_csv(csv_path)

In [16]:
class_names = [
    'anthracnose-cashew', 'bacterial blight-cassava', 'brown spot-cassava',
    'cashew-mosaic-cassava', 'fall armyworm-maize', 'grasshopper-maize',
    'green mite-cassava', 'gumosis-cashew', 'healthy-cashew',
    'healthy-cassava', 'healthy-maize', 'healthy-tomato', 'leaf beetle-maize',
    'leaf blight-maize', 'leaf blight-tomato', 'leaf curl-tomato',
    'leaf miner-cashew', 'leaf spot-maize', 'red rust-cashew',
    'septoria leaf spot-tomato', 'streak virus-maize', 'verticillium wilt-tomato'
]

# Function to preprocess the uploaded image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(150, 150))  # Resize to match model input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the image
    return img_array

# Function to predict the pest/disease class
def predict_pest(image_path):
    img = preprocess_image(image_path)
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions, axis=1)  # Get the index of the class with the highest probability
    return predicted_class[0]

# Function to map the predicted class to a solution from the CSV
def get_solution_for_class(class_index):
    class_name = class_names[class_index]

    # Assuming your CSV has 'Class' column for the class names and 'Solution' column for corresponding solutions
    solution_row = solutions_df[solutions_df['Diseases'] == class_name]
    if not solution_row.empty:
        solution = solution_row['OrganicPrevention'].values[0]
        return solution
    else:
        return "Solution not found for this class."

# Function to process user-uploaded image and return class and solution
def process_user_image(image_path):
    if os.path.exists(image_path):
        # Predict the pest/disease class
        predicted_class = predict_pest(image_path)

        # Get the corresponding pest/disease name
        class_name = class_names[predicted_class]

        # Get the solution for the predicted class
        solution = get_solution_for_class(predicted_class)

        result = {
            "predicted_class": class_name,
            "suggested_solution": solution
        }

        return result
    else:
        return {"error": "Image not found. Please upload a valid image."}



In [17]:
# Example usage: replace 'user_uploaded_image.jpg' with the actual path of the uploaded image
image_path = '/content/ant_cashew.png'  # Replace this with the actual user-uploaded image path
result = process_user_image(image_path)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
{'predicted_class': 'anthracnose-cashew', 'suggested_solution': 'Neem oil spray, Pruning infected branches'}


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 424ms/step


KeyError: 'Class'